In [1]:
import pandas as pd
#from modules.lavenshtein import lavenshtein, get_laven
from Levenshtein import distance
import os
from multiprocessing import Pool
from tqdm import tqdm
from modules.rev_helper import rev_helper

In [21]:
#gets the training and testing data
users_data = pd.read_csv("data/interim/Users.csv")
test_df = pd.read_csv("data/original/test_transactions.csv")
test_df = users_data[users_data['documentid'].isin(test_df['documentid'].values.tolist())]
users_data = users_data[~users_data['documentid'].isin(test_df['documentid'].values.tolist())]

df = pd.concat([test_df, users_data], ignore_index=True)
df.to_csv(os.path.join('data/interim/',"Users.csv"))

#gets the testing documentids
reciepts_to_use = test_df['documentid'].values.tolist()


['00d0472780579', '00d0269258457', '00d0145054354', '00d0795468800', '00d0682248746', '00d0471112949', '00d0472591457', '00d0354593861', '00d0179226173', '00d0277307035', '00d0902256238', '00d0230547657', '00d0859401532', '00d0343958098', '00d0676711962', '00d0114666542', '00d0507143553', '00d0622420051', '00d0831446713', '00d0472127340', '00d0164203891', '00d0343123955', '00d0714317145', '00d0443302756', '00d0484705589', '00d0415751725', '00d0726056883', '00d0916006350', '00d0660852423', '00d0811301203', '00d0741622526', '00d0204688228', '00d0115214912', '00d0508747957', '00d0547495073', '00d0504003816', '00d0870184164', '00d0128862298', '00d0658523473', '00d0537233731', '00d0384888865', '00d0564199588', '00d0207352920', '00d0121860229', '00d0729993616', '00d0950145735', '00d0777749051', '00d0802473715', '00d0261309649', '00d0178057064', '00d0583684114', '00d0403823907', '00d0695919511', '00d0374072400', '00d0689877704', '00d0184132278', '00d0547632877', '00d0290184065', '00d055237557

In [28]:
#cleaning training data
'''
iterates through all the ocr files an finds the Levenshtein distance between the file's 
text content and the provided user data.
outputs a dataframe with the documentid, a list of potential userids, and a list of their
corresponding Levenshtein distances
'''
columns = ["BB1", "BB2", "BB3", "BB4", "BB5", "BB6", "BB7", "BB8", "Text_Main"]
columns_labels = ["documentid","pot_doc","pot_name","pot_price","pot_date","pot_address",]
ocrdir = 'data/interim/ocr/'
levan_user = pd.DataFrame(columns=columns_labels)
#reads user data
users_data = pd.read_csv("data/interim/Users.csv")
#removes all files that are not present in the user file
file_names = [os.path.splitext(file)[0] for file in list(os.listdir(ocrdir))]
file_names = [x for x in file_names if x in users_data['documentid'].values]
file_names = [file_name + '.csv' for file_name in file_names]
#pools the function so that it runs in paralell
with Pool() as p:
    q = p.map(rev_helper, zip(file_names, [users_data for _ in range(len(os.listdir(ocrdir)))]))
for qq in q:
    levan_user.loc[len(levan_user)] = qq

df = levan_user

'''
removes the element in the list at the specified position for all columns
'''
def remove_element_at_position(value):
    #if there is no specified position, remove nothing
    if not value['position']:
        return pd.Series({'documentid':value['documentid'],'pot_doc':value['pot_doc'],'pot_name':value['pot_name'], 'pot_price':value['pot_price'],'pot_address':value['pot_address'],'pot_date':value['pot_date']})
    pot_doc = [value['pot_doc'][i] for i in value['position']]
    pot_name = [value['pot_name'][i] for i in value['position']]
    pot_price = [value['pot_price'][i] for i in value['position']]
    pot_address = [value['pot_address'][i] for i in value['position']]
    pot_date = [value['pot_date'][i] for i in value['position']]
    return pd.Series({'documentid':value['documentid'],'pot_doc':pot_doc,'pot_name':pot_name, 'pot_price':pot_price,'pot_address':pot_address,'pot_date':pot_date})

#finds the position of the ones in the pot date column
df['position'] = df['pot_date'].apply(lambda x: [i for i, x in enumerate(x) if x == 1])

df = df.apply(remove_element_at_position, axis=1)

#finds the position of the ones in the pot price column
df['position'] = df['pot_price'].apply(lambda x: [i for i, x in enumerate(x) if x == 1])

df = df.apply(remove_element_at_position, axis=1)
display(df)
df.to_csv(os.path.join('data/final/', "final_training.csv"))


,documentid,pot_doc,pot_name,pot_price,pot_address,pot_date
0,00d0243046961,[00d0243046961],[9],[1],[93],[1]
1,00d0164592408,[00d0164592408],[1],[0],[92],[1]
2,00d0696518680,[00d0696518680],[11],[1],[97],[1]
3,00d0142736377,[00d0142736377],[19],[1],[93],[1]
4,00d0530724847,[00d0530724847],[1],[1],[96],[1]
...,...,...,...,...,...,...
620,00d0354051709,[00d0354051709],[11],[1],[95],[1]
621,00d0269258457,[00d0840498429],[32],[1],[96],[0]
622,00d0726523245,[00d0726523245],[6],[0],[93],[1]
623,00d0318453525,[00d0318453525],[11],[0],[100],[1]


In [29]:
#cleaning testing data
'''
iterates through all the ocr files an finds the Levenshtein distance between the file's 
text content and the provided user data.
outputs a dataframe with the documentid, a list of potential userids, and a list of their
corresponding Levenshtein distances
'''
columns = ["BB1", "BB2", "BB3", "BB4", "BB5", "BB6", "BB7", "BB8", "Text_Main"]
columns_labels = ["documentid","pot_doc","pot_name","pot_price","pot_date","pot_address",]
ocrdir = 'data/interim/ocr/'
levan_user = pd.DataFrame(columns=columns_labels)
#reads user data
users_data = pd.read_csv("data/interim/Users.csv")
#removes all files that are not present in the user file
file_names = reciepts_to_use
file_names = [file_name + '.csv' for file_name in file_names]
#pools the function so that it runs in paralell
with Pool() as p:
    q = p.map(rev_helper, zip(file_names, [users_data for _ in range(len(os.listdir(ocrdir)))]))
for qq in q:
    levan_user.loc[len(levan_user)] = qq

df = levan_user

'''
removes the element in the list at the specified position for all columns
'''
def remove_element_at_position(value):
    if not value['position']:
        return pd.Series({'documentid':value['documentid'],'pot_doc':value['pot_doc'],'pot_name':value['pot_name'], 'pot_price':value['pot_price'],'pot_address':value['pot_address'],'pot_date':value['pot_date']})
    pot_doc = [value['pot_doc'][i] for i in value['position']]
    pot_name = [value['pot_name'][i] for i in value['position']]
    pot_price = [value['pot_price'][i] for i in value['position']]
    pot_address = [value['pot_address'][i] for i in value['position']]
    pot_date = [value['pot_date'][i] for i in value['position']]
    return pd.Series({'documentid':value['documentid'],'pot_doc':pot_doc,'pot_name':pot_name, 'pot_price':pot_price,'pot_address':pot_address,'pot_date':pot_date})
        
#finds the position of the ones in the pot date column
df['position'] = df['pot_date'].apply(lambda x: [i for i, x in enumerate(x) if x == 1])

df = df.apply(remove_element_at_position, axis=1)

#finds the position of the ones in the pot price column
df['position'] = df['pot_price'].apply(lambda x: [i for i, x in enumerate(x) if x == 1])

df = df.apply(remove_element_at_position, axis=1)

display(df)

df.to_csv(os.path.join('data/final/', "final_test.csv"))

,documentid,pot_doc,pot_name,pot_price,pot_address,pot_date
0,00d0472780579,[00d0472780579],[1],[1],[96],[1]
1,00d0269258457,[00d0840498429],[32],[1],[96],[0]
2,00d0145054354,[00d0145054354],[0],[1],[93],[1]
3,00d0795468800,[00d0795468800],[2],[1],[96],[1]
4,00d0682248746,[00d0682248746],[11],[1],[96],[0]
...,...,...,...,...,...,...
121,00d0413686396,[00d0413686396],[14],[0],[93],[1]
122,00d0745331693,"[00d0729993616, 00d0184132278, 00d0215894408, ...","[2, 2, 2, 0, 2, 2, 2, 0, 0, 16, 2, 13, 2, 0, 14]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[100, 95, 95, 100, 95, 100, 100, 100, 95, 82, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
123,00d0978814485,"[00d0978814485, 00d0920899195]","[11, 11]","[0, 0]","[95, 95]","[0, 0]"
124,00d0902251681,[00d0902251681],[18],[0],[92],[1]
